In [1]:
from deap import base, creator, tools
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

全局参数

In [ ]:
TMAX = 500          # 为演示缩短长度，实际可设1000
K = 10
P = 3

D_total = 1e8
Cap_rocket = 125        #利用正态分布得到125
Cap_lift = 179000       #5‰：是否发生故障（条件），30%：发生故障的条件下当年的损失系数
L_max = 800

Cost_rocket = 500000        #1.5%：是否发射失败（条件），{0.7(火箭概率)*火箭发射单价/吨；0.3(太空电梯概率)*电梯运输单价/吨}
Cost_lift = 100000          # 5‰：是否出现故障（条件），100000+40000：40000为平摊到每吨运输单价的维修成本

电梯运载量与火箭/太空电梯每吨发射成本的随机选择函数，返回值均为int

In [3]:
def randomElevCapability(TMAX, Cap_lift, P_Efail=0.005):
    Cap_lift_t = np.zeros(TMAX)

    for t in range(TMAX):
        if random.random() < P_Efail:
            # 故障年份：运力下降 60%~85%
            Cap_lift_t[t] = Cap_lift * random.uniform(0.6, 0.85)
        else:
            # 正常年份：轻微波动 ±5%
            Cap_lift_t[t] = Cap_lift * random.uniform(0.95, 1.05)

    return Cap_lift_t


def randomRocketCost(TMAX,
                     base_cost=62500000,
                     launches_per_year=8000,
                     P_Rfail=0.015,
                     severe_ratio=0.3):
    
    Cost_rocket_t = np.zeros(TMAX)

    for t in range(TMAX):
        # 失败次数
        failures = np.random.binomial(launches_per_year, P_Rfail)

        # 严重失败次数
        severe_failures = np.random.binomial(failures, severe_ratio)

        # 成本计算
        base_total = base_cost * launches_per_year
        normal_loss = failures * base_cost
        severe_loss = severe_failures * (base_cost * 0.2)  # 严重损失 20%

        avg_cost = (base_total + normal_loss + severe_loss) / launches_per_year

        # 正常年份轻微浮动
        avg_cost *= random.uniform(0.97, 1.03)

        Cost_rocket_t[t] = avg_cost

    return Cost_rocket_t


def randomElevCost(TMAX, Cost_lift, P_Efail=0.005):
    Cost_lift_t = np.zeros(TMAX)

    for t in range(TMAX):
        if random.random() < P_Efail:
            # 故障年份：上涨 20%~60%
            Cost_lift_t[t] = Cost_lift * random.uniform(1.2, 1.6)
        else:
            # 正常年份：±5%
            Cost_lift_t[t] = Cost_lift * random.uniform(0.95, 1.05)

    return Cost_lift_t


    


使用Monte Carlo法生成3*TMAX的时间过程数据，返回值是字典

In [4]:
def generate_MC_parameters(TMAX, seed=None):
    """
    生成一次 Monte Carlo 所需的参数矩阵
    返回 dict便于后续扩展
    """
    if seed is not None:
        np.random.seed(seed)

    # ==============================
    # 占位：参数矩阵初始化
    # ==============================

    Cap_lift_t = np.zeros(TMAX)
    Cost_rocket_t = np.zeros(TMAX)
    Cost_lift_t = np.zeros(TMAX)

    # ==============================
    # 约束规则

    Cap_lift_t = randomElevCapability(TMAX, Cap_lift)
    Cost_rocket_t = randomRocketCost(TMAX)
    Cost_lift_t = randomElevCost(TMAX, Cost_lift)
    
    # ==============================

    return {
        "Cap_lift": Cap_lift_t,
        "Cost_rocket": Cost_rocket_t,
        "Cost_lift": Cost_lift_t
    }


初始化DEAP

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 火箭发射次数 (整数)
toolbox.register("attr_R", lambda: random.randint(0, L_max))
# 电梯运输量 (连续)
toolbox.register("attr_E", lambda: random.uniform(0, Cap_lift))

def init_individual():
    """保证初始化总运输量大约 D_total / TMAX"""
    ind = []
    avg_rocket_per_year = D_total / TMAX / 2 / Cap_rocket / K
    avg_lift_per_year = D_total / TMAX / 2 / P

    for _ in range(K*TMAX):
        val = max(1, int(random.gauss(avg_rocket_per_year, avg_rocket_per_year*0.5)))
        ind.append(min(val, L_max))

    for _ in range(P*TMAX):
        val = random.gauss(avg_lift_per_year, avg_lift_per_year*0.5)
        val = max(0, min(val, Cap_lift))
        ind.append(val)

    return creator.Individual(ind)

toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


考虑时间随机过程的fitness函数

In [6]:
def assign_fitness_MC(pop, params, w1=0.5, w2=0.5):
    """
    Monte Carlo 版本 fitness
    params: dict, 来自 generate_MC_parameters
    """
    Cap_lift_t = params["Cap_lift"]
    Cost_rocket_t = params["Cost_rocket"]
    Cost_lift_t = params["Cost_lift"]

    ZC_list = []
    ZT_list = []

    for ind in pop:
        R = np.array(ind[:K*TMAX]).reshape(K, TMAX)
        E = np.array(ind[K*TMAX:]).reshape(P, TMAX)        #解压染色体

        # ==============================
        # 总成本（逐年）
        # ==============================
        ZC = 0.0
        for t in range(TMAX):
            ZC += Cost_rocket_t[t] * np.sum(R[:, t])
            ZC += Cost_lift_t[t] * np.sum(E[:, t])

        # ==============================
        # 总工期（逐年容量）
        # ==============================
        transported = 0.0
        ZT = TMAX

        for t in range(TMAX):
            R_t = np.sum(R[:, t]) * Cap_rocket  
            E_t = np.sum(E[:, t])
            E_t = min(E_t, Cap_lift_t[t])       # 年度约束

            transported += R_t + E_t

            if transported >= D_total:
                ZT = t + 1
                break

        # ==============================
        # 罚函数接口
        # ==============================
        penalty = abs(transported - D_total) / D_total * 1e-6

        ind.ZC = ZC
        ind.ZT = ZT
        ind.penalty = penalty

        ZC_list.append(ZC)
        ZT_list.append(ZT)

    # ==============================
    # 种群标准化
    # ==============================
    std_C = np.std(ZC_list) + 1e-6
    std_T = np.std(ZT_list) + 1e-6

    for ind in pop:
        fitness_val = w1 * ind.ZC / std_C + w2 * ind.ZT / std_T + ind.penalty
        ind.fitness.values = (fitness_val,)


遗传算子

In [7]:
toolbox.register("mate", tools.cxTwoPoint)

def mutate_bound(ind, mu_R=0, sigma_R=100, mu_E=0, sigma_E=5e4, indpb=0.1):
    for i in range(len(ind)):
        if random.random() < indpb:
            if i < K*TMAX:
                ind[i] += int(round(random.gauss(mu_R, sigma_R)))
                ind[i] = max(0, min(L_max, ind[i]))
            else:
                ind[i] += random.gauss(mu_E, sigma_E)
                ind[i] = max(0, min(Cap_lift, ind[i]))
    return ind,

toolbox.register("mutate", mutate_bound)
toolbox.register("select", tools.selTournament, tournsize=3)

Monte Carlo + GA主函数，返回值为一个列表（列表元素均为字典），其中保留了每一个随机过程得到最优解的染色体信息，ZC，ZT与随机过程参数

In [8]:
def run_MC_GA(N_MC):
    best_solutions = []

    for mc in range(N_MC):
        print(f"\n===== Monte Carlo run {mc+1}/{N_MC} =====")

        # 生成一次随机参数
        params = generate_MC_parameters(TMAX, seed=mc)

        # 初始化种群
        pop = toolbox.population(n=50)
        NGEN = 500
        CXPB = 0.7
        MUTPB = 0.2

        assign_fitness_MC(pop, params)

        for gen in range(NGEN):
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))

            for c1, c2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < CXPB:
                    toolbox.mate(c1, c2)
                    del c1.fitness.values, c2.fitness.values

            for ind in offspring:
                if random.random() < MUTPB:
                    toolbox.mutate(ind)
                    del ind.fitness.values

            assign_fitness_MC(offspring, params)
            pop[:] = offspring
        
        best = tools.selBest(pop, 1)[0]

        # ========================================================
        # --- 修复潜在污染问题 ---
        # 使用局部变量，不覆盖全局 K, P, TMAX
        K_local, T_local = K, TMAX
        P_local = P

        # 计算每年总运输量时使用 Monte Carlo 参数
        Cap_lift_t = params["Cap_lift"]
        Cost_rocket_t = params["Cost_rocket"]
        Cost_lift_t = params["Cost_lift"]

        R = np.array(best[:K_local*T_local]).reshape(K_local, T_local)
        E = np.array(best[K_local*T_local:]).reshape(P_local, T_local)

        # 计算每年总运输量，考虑 MC 电梯上限
        yearly_transport = np.array([
            R[:, t].sum() * Cap_rocket + min(E[:, t].sum(), Cap_lift_t[t])
            for t in range(T_local)
        ])

        # 按年份运输量从大到小排序
        sorted_idx = np.argsort(-yearly_transport)

        # 初始化新的矩阵
        R_new = np.zeros_like(R)
        E_new = np.zeros_like(E)

        transported = 0
        ZT_new = 0

        for t_new, t_old in enumerate(sorted_idx):
            # 尽量保持原有分配比例，但不超过上限
            R_year = np.minimum(R[:, t_old], L_max)
            E_year = np.minimum(E[:, t_old], Cap_lift_t[t_old])  # 使用 MC 参数

            # 计算这一年的运输量
            transported += R_year.sum() * Cap_rocket + E_year.sum()
            
            # 如果运输量超过 D_total，则按比例缩减这一年的运输量
            if transported > D_total:
                excess = transported - D_total
                total_year = R_year.sum() * Cap_rocket + E_year.sum()
                if total_year > 0:
                    ratio = (total_year - excess) / total_year
                    R_year = np.floor(R_year * ratio)
                    E_year = E_year * ratio
                transported = D_total
                ZT_new = t_new + 1
                R_new[:, t_new] = R_year
                E_new[:, t_new] = E_year
                break

            R_new[:, t_new] = R_year
            E_new[:, t_new] = E_year
            ZT_new = t_new + 1

        # 重新计算压缩后的总成本，使用 Monte Carlo 参数
        ZC_new = 0
        for t in range(ZT_new):
            ZC_new += Cost_rocket_t[t] * R_new[:, t].sum()
            ZC_new += Cost_lift_t[t] * E_new[:, t].sum()

        # ========================================================
        
        
        
        # 保存结果时复制染色体，避免引用污染
        best_solutions.append({
            "chromosome": list(best),
            "ZC": best.ZC,
            "ZT": best.ZT,
            "ZT_zip": ZT_new,
            "ZC_zip": ZC_new,
            "ROCKET_zip": R_new.sum() * Cap_rocket,
            "ELEVATOR_zip": E_new.sum(),
            "params": params
        })

    return best_solutions


运行GA

In [9]:
opt_ss = run_MC_GA(1)      #<————改变这个参数以增减蒙特卡洛采样数

for best in range(len(opt_ss)):
    W = opt_ss[best]['ZC'] * 0.5 + opt_ss[best]['ZT'] * 0.5
    print(f"\n>>> 第{best+1}个随机点的W={W}, ZC={opt_ss[best]['ZC']:.3e}, ZT={opt_ss[best]['ZT']:.3e} <<<")
    print(f"\n<<< 第{best+1}个随机点压缩后的pZT={opt_ss[best]['ZT_zip']}, pZC={opt_ss[best]['ZC_zip']}, ROCKET={opt_ss[best]['ROCKET_zip']}, ELEV={opt_ss[best]['ELEVATOR_zip']} >>>")
    print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")






===== Monte Carlo run 1/1 =====

>>> 第1个随机点的W=5278751820098.502, ZC=1.056e+13, ZT=5.000e+02 <<<

<<< 第1个随机点压缩后的pZT=500, pZC=10556761558073.578, ROCKET=10826250, ELEV=50514828.43313214 >>>

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
